# Fine Tuning

* `beomi/kcbert-base` 모델에 올리브영 상품 리뷰 문장과 Sentiment 간의 패턴을 fine-tuning합니다.

* 두 개의 fine-tuned 모델을 HuggingFace Hub에 저장하여 운용합니다.

In [1]:
!pip install --upgrade pandas transformers datasets accelerate evaluate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 72.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 68.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 166.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 135.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 164.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [12]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, DatasetDict, ClassLabel
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding
)
import evaluate

In [3]:
df = pd.read_csv('labeled_sample.csv', encoding='utf-8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     2000 non-null   object
 1   성분      2000 non-null   object
 2   별점      2000 non-null   object
 3   피부타입    2000 non-null   object
 4   피부고민    2000 non-null   object
 5   자극도     2000 non-null   object
 6   문장      2000 non-null   object
 7   label   2000 non-null   object
dtypes: object(8)
memory usage: 125.1+ KB


In [4]:
labels = sorted(df['label'].unique().tolist())
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}
df['labels'] = df['label'].map(label2id)

In [5]:
df['labels']

0       0
1       0
2       0
3       1
4       0
       ..
1995    0
1996    1
1997    2
1998    2
1999    2
Name: labels, Length: 2000, dtype: int64

In [30]:
dataset = Dataset.from_pandas(df)

class_label_feature = ClassLabel(num_classes=len(labels), names=labels)
dataset = dataset.cast_column("labels", class_label_feature)

train_eval_dataset = dataset.train_test_split(test_size=0.1, stratify_by_column='labels')

dd = DatasetDict({
    'train': train_eval_dataset['train'],
    'eval': train_eval_dataset['test']
})

print(dd)

Casting the dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['제품명', '성분', '별점', '피부타입', '피부고민', '자극도', '문장', 'label', 'labels'],
        num_rows: 1800
    })
    eval: Dataset({
        features: ['제품명', '성분', '별점', '피부타입', '피부고민', '자극도', '문장', 'label', 'labels'],
        num_rows: 200
    })
})


In [31]:
model = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model)

In [32]:
def tokenize_function(examples):
    return tokenizer(examples["문장"], padding=False, truncation=True, max_length=128)

tokenized_datasets = dd.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['문장', 'label'])


Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [33]:
model = AutoModelForSequenceClassification.from_pretrained(
    model,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [35]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    
    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}

In [36]:
training_args = TrainingArguments(
    output_dir="./kcbert_full_finetuned", 
    learning_rate=2e-5,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

In [37]:
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/tmp/ipykernel_325/3622241889.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [38]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.568441,0.775000,0.754612
2,No log,0.530516,0.775000,0.760908
3,No log,0.638956,0.815000,0.801893
4,No log,0.740763,0.790000,0.783022
5,0.338300,0.754495,0.795000,0.790341


TrainOutput(global_step=565, training_loss=0.30390086638188996, metrics={'train_runtime': 32.6555, 'train_samples_per_second': 275.604, 'train_steps_per_second': 17.302, 'total_flos': 203740230558816.0, 'train_loss': 0.30390086638188996, 'epoch': 5.0})

In [39]:
best_model_path = "./best_kcbert_full_model_2"
trainer.save_model(best_model_path)

In [40]:
def predict_sentiment(text):
    inputs = loaded_tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = loaded_model(**inputs)
    
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return loaded_model.config.id2label[prediction]

In [26]:
test_case = pd.read_csv('tokenized/tokenized_essence_31_35.csv')
test_case = list(test_case['문장'].sample(50).items())
test_case

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xec in position 0: unexpected end of data

In [41]:
test_case = ['이게 막 번들번들한 그런느낌이 아니라 정말 촉촉한 느낌 그대로 유지가 되기때문에 메이크업도 잘 먹구요!',
 ':)\n\n사용한지 오래 되지 않았지만\n그 동안 사용해보았던 미백 제품들 중에서는\n단연 제일 맘에 쏙 들어요!!!!!',
 '향이 너무 상큼하구 잡티+미백+탄력 신경쓰이시는 분들에게 딱인것 같습니다 ㅎㅎ',
 '이렇게 가격이 저렴한 앰플은 처음봐요',
 '전 앞으로 이것만 쓰려구용ㅠㅠ\n첨에 인스타 광고 보고 속는셈치고 사봤는데용..',
 '많이 바르면 끈적이기는 하지만 한 두방울 섞어서 바르면 건조함을 좀 잡아줘요',
 '감사합니가 너무 이뻐요',
 '리들샷 평소에 잘 써서 같이 써보려구 구매햇어용',
 '그외에 봄 가을 겨울 사용하기 너무 좋아요~',
 '촉촉하고 쫀쫀하게 마무리되네요.',
 '저 지성도 건성도 아닌 애매한 사람인데 이거 쫀쫀한 타입이지만 번들거리거나 무겁지 않고 수분감 채워줘서 너무 좋아용 저같은 분들 사용해보세오',
 '제형은 가벼운 것 같으나 약간 꾸덕한 느낌을 가지고 있어서 흘러내리거나 하진 않고 발림성 좋습니다.',
 '비타민c 제품이라 차광되는 병때문에 얼마나 남았는지 모르는게 단점이라면 단점이네요ㅠ',
 '큰 기대 하지 않고 샀는데 정말 맘에 드는 제품입니다 ~',
 '20대피부로 돌아갈 수는 없지만 내 나이보다는 젊어보여야하잖아요~',
 '그래도 앰플이 꾸덕하거나 너무 기름지지 않고 그렇다고 또 너무 산뜻하지도 않고 보습감은 있으면서 유분감은 없는것 같아요!',
 '특히 겨울 보습관리네 좋은 것 같습니다',
 '잡티 앰플은 자극적인 경우가 많았는데 미백, 잡티 앰플임에도 자극이 없이 순해서 좋아요.',
 '두통째 사용했습니다!!',
 '그리고 민감 피부도 안심하고 사용할 수 있는 저자극 에센스라 좋아요.',
 '맑아진 제 피부를 꿈꾸며 써보겠습니다 ! ㅎㅎ',
 '흐르는 제형이라 더 수분감이 많이 느껴지는 것 같네요',
 '좁쌀이랑 다 들어갔어요',
 '너무 금방 쓰거든요. ㅜㅜ',
 '끈적한거 엄청 싫어하는데 바르면 실크처럼 부드럽게\n발려서 너무 좋아요!!',
 '일단 공식홈페이지에서는 7만9천원에 판매하는데\n<나이아신아마이드 아데노신>\n이 두 가지만 들어간 제품이 왜 그리도 비싼 건지..?',
 '다른 제품을 같이 써서인지는\n모르겠으나, 어느날부터 광채가 생긴 것 같아요.',
 '발림성 - 적당한 발림성.\n＊',
 '가격이 비싸다고 생각했지만 2달 정도 쓰고 보니까 합리적이라는 생각이 들어요.',
 '오래가진 않지만 반짝거려요👍',
 '바르면 수분이 차오른다는 말이 맞는거 같아요',
 '피부가 건성인분들은 이 제품을 꼭 추천해드리고 싶고 트라이한번 해보세요',
 '자극이 없고 밀려나오는 제형이 아닌 물제형이라 발림성이 좋네요.',
 '피부가 쫙 펴지고 탱탱해진 게 딱 느껴졌어요ㅠㅠ',
 '향 - 저는 향에 좀 예민한 편인데, 호불호없는 향.',
 '여태껏 인생 써본 앰플중에 최고임',
 '흡수는 꽤 잘되는편입니다.',
 '그리고 무엇보다 홍조가 있는편인데 피부가 진정이 되어서 붉은기도 많이 좋아지고 있네요.',
 '시간이 지남에 따라 제형변화가 있어서 6개월안에 사용하는걸 추천합니다.',
 '보습 빵빵해요!',
 '향은 플로럴향이고, 바르면 수분감 채워주면 엄청 약간의 유분기?가 느껴집니다.',
 '한통 다쓰고 재구매 했습니다',
 '근데 세상 매끄럽고\n더 꾸덕하고 촉촉해요.',
 '여름이라 한두번사용했어요',
 '요즘 자외선 많이 받다보니까 잡티가 올라오는 느낌이라\n너무 고민이었거든요',
 '미샤 비타c잡티 앰플 좋아요',
 '그리고 구달 청귤세럼 굉장히 잘 사용했지만, 최근 사용했을 때 뭔가 젤같은 느낌때문에 다음단계 바르기가 좀 애매하다고 느꼈던 적이 있었어요.',
 '사실 좀 신생 브랜드라 좀 꺼려졋긴한데 제가 한번 써보고 한달뒤에 효과가 어땟는지 리뷰쓰러 올께요',
 '이거 완전 수분감이 엄청나서 아주 맘에 들어요~',
 '세안하자마자 물기만 닦고 바로 바르는데 피부가 촉촉하고 쫀쫀해져요.']

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 저장된 모델과 토크나이저를 메모리로 불러옵니다.
# 이 과정에서 loaded_model과 loaded_tokenizer 변수가 생성됩니다.
try:
    loaded_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)
    loaded_tokenizer = AutoTokenizer.from_pretrained(best_model_path)
    loaded_model.to(device) # 모델을 GPU로 이동
    print(f"'{best_model_path}'에서 모델과 토크나이저를 성공적으로 로드했습니다.")
    print(f"사용 장치: {device}")
except OSError:
    print(f"오류: '{best_model_path}' 경로에 모델 파일이 없습니다. 경로를 다시 확인해주세요.")


# --- 2. 추론 함수 정의 ---

# loaded_model과 loaded_tokenizer를 사용하는 함수를 다시 한번 정의해줍니다.
def predict_sentiment(text):
    # 함수 내에서 NameError가 발생하지 않도록, 필요한 변수들이 이 코드 블록 위에 모두 정의되어 있어야 합니다.
    inputs = loaded_tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = loaded_model(**inputs)
    
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return loaded_model.config.id2label[prediction]


'./best_kcbert_full_model_2'에서 모델과 토크나이저를 성공적으로 로드했습니다.
사용 장치: cuda


In [44]:
for sentence in test_case:
    predicted_label = predict_sentiment(sentence)
    
    print(f"문장: {sentence}")
    print(f"  -> 예측 결과: {predicted_label}\n")

문장: 이게 막 번들번들한 그런느낌이 아니라 정말 촉촉한 느낌 그대로 유지가 되기때문에 메이크업도 잘 먹구요!
  -> 예측 결과: 긍정

문장: :)

사용한지 오래 되지 않았지만
그 동안 사용해보았던 미백 제품들 중에서는
단연 제일 맘에 쏙 들어요!!!!!
  -> 예측 결과: 긍정

문장: 향이 너무 상큼하구 잡티+미백+탄력 신경쓰이시는 분들에게 딱인것 같습니다 ㅎㅎ
  -> 예측 결과: 긍정

문장: 이렇게 가격이 저렴한 앰플은 처음봐요
  -> 예측 결과: 긍정

문장: 전 앞으로 이것만 쓰려구용ㅠㅠ
첨에 인스타 광고 보고 속는셈치고 사봤는데용..
  -> 예측 결과: 긍정

문장: 많이 바르면 끈적이기는 하지만 한 두방울 섞어서 바르면 건조함을 좀 잡아줘요
  -> 예측 결과: 중립

문장: 감사합니가 너무 이뻐요
  -> 예측 결과: 긍정

문장: 리들샷 평소에 잘 써서 같이 써보려구 구매햇어용
  -> 예측 결과: 긍정

문장: 그외에 봄 가을 겨울 사용하기 너무 좋아요~
  -> 예측 결과: 긍정

문장: 촉촉하고 쫀쫀하게 마무리되네요.
  -> 예측 결과: 긍정

문장: 저 지성도 건성도 아닌 애매한 사람인데 이거 쫀쫀한 타입이지만 번들거리거나 무겁지 않고 수분감 채워줘서 너무 좋아용 저같은 분들 사용해보세오
  -> 예측 결과: 긍정

문장: 제형은 가벼운 것 같으나 약간 꾸덕한 느낌을 가지고 있어서 흘러내리거나 하진 않고 발림성 좋습니다.
  -> 예측 결과: 부정

문장: 비타민c 제품이라 차광되는 병때문에 얼마나 남았는지 모르는게 단점이라면 단점이네요ㅠ
  -> 예측 결과: 부정

문장: 큰 기대 하지 않고 샀는데 정말 맘에 드는 제품입니다 ~
  -> 예측 결과: 긍정

문장: 20대피부로 돌아갈 수는 없지만 내 나이보다는 젊어보여야하잖아요~
  -> 예측 결과: 긍정

문장: 그래도 앰플이 꾸덕하거나 너무 기름지지 않고 그렇다고 또 너무 산뜻하지도 않고 보습감은 있으면서 유분감은 없는것 같아요!
  -> 예측 결과: 긍정

문장: 

In [45]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
checkpoint_path = './kcbert_full_finetuned/checkpoint-339'

from transformers import TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer

model_id = 'iPad7/kcbert-base-rsa-e3'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

model.push_to_hub(model_id, commit_message='Pushed model for epoch 3 model')
tokenizer.push_tu_hub(model_id, commit_message='Pushed tokenizer for epoch 3 model')

# trainer.push_to_hub(commit_message="Initial model training complete")
print(f"모델이 '{model_id}' 저장소에 성공적으로 푸시되었습니다.")

Uploading...:   0%|          | 0.00/436M [00:00<?, ?B/s]